In [ ]:
import numpy as np
import onnxruntime as ort
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.onnx
from onnxruntime.quantization import QuantType, quantize_dynamic

import onnx


### float32→int8

In [2]:
MIN=-200 # データの最小値
MAX=412 # データの最大値

S = (255/(MAX-MIN)) # scale

_MIN = MIN*S
_MAX = MAX*S
_MIN,_MAX  # -127~128の範囲に入っていないのでzero-pointで調整

(-83.33333333333334, 171.66666666666669)

In [5]:
# zero-point
print(f"{np.abs(-128-_MIN)},{np.abs(127-_MAX)}")
print(f"{round(np.abs(-128-_MIN))},{round(np.abs(127-_MAX))}")

44.66666666666666,44.666666666666686
45,45


126.66666666666669